In [1]:
from loader1 import *
from utils import *

import warnings
warnings.filterwarnings('ignore')

In [3]:
_ = load_all()

Some files already exist in your CACHE_DIR. If you still want to run this function,              run with override=True


In [4]:
gps = read_data('gps')
orders = read_data('order')

In [5]:
gps.shape, orders.shape

((32155517, 5), (209423, 7))

In [6]:
# Run util functions
orders = convert_unix_ts(orders, timecols=['ride_start_timestamp', 'ride_stop_timestamp'])
orders = ride_duration(orders)

In [10]:
# Add driverid in orders column
drivers = gps[['driver_id', 'order_id']].drop_duplicates()
orders = orders.merge(drivers, on='order_id', how='left')

In [11]:
orders.shape

(209423, 9)

In [26]:
orders.head()

,order_id,ride_start_timestamp,ride_stop_timestamp,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,ride_duration,driver_id
0,eb9dd4095d9850e6287cefd813775a6c,2016-11-01 01:46:37,2016-11-01 02:15:07,104.094640,30.703971,104.089270,30.650850,28.500000,0373c98625e6d8e1fc56a4d61318138c
1,387a742fa5a3fbe4a1f215ac58ea33a8,2016-11-01 07:33:05,2016-11-01 08:07:55,104.076509,30.767430,104.063700,30.589510,34.833333,0373c98625e6d8e1fc56a4d61318138c
2,9cf55f8e6e02a1e0f792df06e5d85011,2016-11-01 12:55:52,2016-11-01 13:16:57,104.019699,30.689007,104.105324,30.663950,21.083333,0373c98625e6d8e1fc56a4d61318138c
3,5feeae0307e15203484b9ffceef89855,2016-11-01 08:44:00,2016-11-01 09:04:25,104.036090,30.622690,104.043860,30.682320,20.416667,0373c98625e6d8e1fc56a4d61318138c
4,ad4b52cb15b90c44c8f20a8b1e57a622,2016-10-31 23:53:25,2016-11-01 00:02:57,104.115997,30.652313,104.104421,30.695113,9.533333,0373c98625e6d8e1fc56a4d61318138c


In [15]:
orders.groupby('driver_id')[['ride_duration']].sum().reset_index()

,driver_id,ride_duration
0,0000131d486b69eb77ab6e9e7cca9f4c,118.966667
1,000211a97ba2b768ef941593bf1ae1a5,214.900000
2,000770ada4aa5d856b5a766fb6bf7cf1,87.900000
3,0007a380800445346fff19afc4071ce3,23.650000
4,0008c9d2ac91c00ef8b3ef80eba02a06,155.566667
...,...,...
35444,fffae1d5a2ca206c5e2c014287c63960,168.800000
35445,fffb1afe11b2cfd5336b222cdee022eb,122.416667
35446,fffc3f877cce16e491f68d69a2163998,30.483333
35447,fffd264654f1acaa393d36d8fb4d9795,447.000000


In [24]:
orders.ride_start_timestamp[4].week

44